In [1]:
import warnings; warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from time import time
import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt

from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
# from gluonts.dataset.rolling_dataset import generate_rolling_dataset, StepStrategy

In [3]:
path = ".."
df = pd.read_parquet(f"{path}/data/processed/outlier_removed.parquet")

In [4]:
GENERATE_SPEED_ANGLE = False
add_lagged = False
INPUT_WIDTH = 120

train_ratio = 0.93
valid_ratio = 0.02

In [5]:
if add_lagged:
    df["production_48_lagged"] = df.groupby("rt_plant_id").production.shift(48)

# SELECTED_PLANT_ID = df.groupby("rt_plant_id").production.sum().sort_values(ascending=False).index[:1].tolist()
SELECTED_PLANT_ID = df.groupby("rt_plant_id").production.sum().sort_values(ascending=False).index[:94].tolist()
print(SELECTED_PLANT_ID)
weather_cols = [col for col in df.columns if col.startswith(("UGRD", "VGRD"))]
if add_lagged:
    df = df.set_index("forecast_dt")[["rt_plant_id", "production", "production_48_lagged", *weather_cols]]
    df = df.dropna()
else:    
    df = df.set_index("forecast_dt")[["rt_plant_id", "production", *weather_cols]]
df = df[df["rt_plant_id"].isin(SELECTED_PLANT_ID)]# .drop("rt_plant_id", axis=1)

[969, 968, 1518, 1484, 1507, 1508, 1527, 672, 1537, 2180, 1509, 1506, 1511, 2113, 1516, 1761, 2123, 1505, 2140, 1878, 1499, 1977, 1492, 1490, 2184, 1945, 2166, 1487, 2073, 1502, 1525, 1523, 1528, 1928, 1517, 1709, 2326, 1900, 1498, 1929, 1741, 2125, 1504, 1460, 1512, 2114, 2323, 1501, 1489, 1959, 1472, 1787, 1513, 1519, 1737, 2224, 1491, 1494, 1578, 2083, 2374, 2116, 1459, 1514, 1759, 1944, 2112, 757, 2062, 1899, 1781, 2063, 2288, 2070, 2058, 2031, 2040, 1194, 2089, 2050, 1883, 2235, 1524, 1939, 1493, 2291, 1485, 1503, 1488, 1943, 1843, 2104, 749, 1655]


In [6]:
if GENERATE_SPEED_ANGLE:
    for box in ["SW", "NW", "NE", "SE"]:
        df[f"speed_{box}"] = np.sqrt(np.square(df[f"UGRD_80.m.above.ground.{box}"]) + np.square(df[f"VGRD_80.m.above.ground.{box}"]))
        df[f"angle_{box}"] = np.arctan(df[f"UGRD_80.m.above.ground.{box}"] / df[f"VGRD_80.m.above.ground.{box}"])
        
time_indices = sorted(df.index.unique())

train_indices = time_indices[:int(len(time_indices) * train_ratio)]
valid_indices = time_indices[int(len(time_indices) * train_ratio):int(len(time_indices) * (train_ratio + valid_ratio))]
test_indices = time_indices[int(len(time_indices) * (train_ratio + valid_ratio)):]

train_df = df.loc[train_indices, :]
valid_df = df.loc[valid_indices, :]
test_df = df.loc[test_indices, :]

print("Train start and end dates:\t", train_indices[0], "\t", train_indices[-1])
try:
    print("Validation start and end dates:\t", valid_indices[0], "\t", valid_indices[-1])
except:
    pass
print("Test start and end dates:\t", test_indices[0], "\t", test_indices[-1])

Train start and end dates:	 2019-01-24 03:00:00 	 2021-11-11 23:00:00
Validation start and end dates:	 2021-11-12 00:00:00 	 2021-12-03 23:00:00
Test start and end dates:	 2021-12-04 00:00:00 	 2022-01-27 23:00:00


In [7]:
ctx = "gpu" if mx.context.num_gpus() else "cpu"
print(ctx)
PREDICTION_LENGTH = 48
CONTEXT_LENGTH = 120
freq = "1H"
EPOCHS = 30

cpu


In [8]:
import gc; gc.collect()

print(df.memory_usage(deep=True).sum() / 1024 ** 2)
df["rt_plant_id"] = df["rt_plant_id"].astype(np.int16)
for col in df.columns:
    if col not in ["rt_plant_id"]:
        df[col] = df[col].astype(np.float16)

gc.collect()
print(df.memory_usage(deep=True).sum() / 1024 ** 2)

240.49053192138672
98.50837707519531


In [9]:
feature_columns = [col for col in train_df.columns if col not in ["production", "rt_plant_id"]]

def dataset_helper(df_, plant_id=None, is_train=True):
    if plant_id is not None:
        df__ = df_[df_["rt_plant_id"] == plant_id]
        plant_id = str(plant_id)
    else:
        plant_id = str(SELECTED_PLANT_ID)
        df__ = df_
    if not is_train:
        df__ = df__.iloc[-(CONTEXT_LENGTH+PREDICTION_LENGTH):]
    return {
        "target": df__.production,
        "start": df__.index[0],
        "item_id": plant_id,
        "feat_dynamic_real": df__[feature_columns].T
    }

In [10]:
from tqdm import tqdm 

In [11]:
train_ds = ListDataset([dataset_helper(df[df.index <= train_indices[-1]], plant_id=plant_id) 
    for plant_id in tqdm(df.rt_plant_id.unique())], freq=freq)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [00:10<00:00,  8.68it/s]


In [15]:
len(valid_indices) // 24

22

In [16]:
[(i+1)*24 for i in range(len(valid_indices) // 24 -1)]

[24,
 48,
 72,
 96,
 120,
 144,
 168,
 192,
 216,
 240,
 264,
 288,
 312,
 336,
 360,
 384,
 408,
 432,
 456,
 480,
 504]

In [18]:
valid_ds = ListDataset(
    [dataset_helper(
        df[df.index < valid_indices[date_shift]], 
        plant_id=plant_id, 
        is_train=False)
        for plant_id in tqdm(df.rt_plant_id.unique())
        for date_shift in [(i+1)*24 for i in range(len(valid_indices) // 24 - 1)]
    ], freq=freq)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [03:40<00:00,  2.35s/it]


In [19]:
gc.collect()

18

In [21]:
test_ds = ListDataset([dataset_helper(
    df[df.index < test_indices[date_shift]], 
    plant_id=plant_id,
    is_train=False) 
    for plant_id in tqdm(df.rt_plant_id.unique())
    for date_shift in [(i+1)*24 for i in range(len(test_indices) // 24 - 1)]
], freq=freq)




  0%|                                                                                                                                                                                       | 0/94 [00:00<?, ?it/s]

  1%|█▊                                                                                                                                                                             | 1/94 [00:06<09:40,  6.24s/it]

  2%|███▋                                                                                                                                                                           | 2/94 [00:12<09:33,  6.23s/it]

  3%|█████▌                                                                                                                                                                         | 3/94 [00:18<09:32,  6.29s/it]

  4%|███████▍                                                                                                                                     

In [22]:
from gluonts.mx.distribution import NegativeBinomialOutput, GaussianOutput, StudentTOutput

mx.random.seed(7)
np.random.seed(7)
deepar_estimator = DeepAREstimator(
    freq=freq, 
    num_layers=2,
    num_cells=80,
    prediction_length=PREDICTION_LENGTH, 
    context_length=CONTEXT_LENGTH, 
    use_feat_dynamic_real=True, 
    trainer=Trainer(epochs=20, ctx=ctx), # learning_rate=1e-3, hybridize=False, num_batches_per_epoch=20
    # distr_output=GaussianOutput()
    # distr_output=NegativeBinomialOutput()
    # distr_output=StudentTOutput()
)
predictor = deepar_estimator.train(training_data=train_ds, validation_data=valid_ds, num_workers=None)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.80it/s, epoch=1/20, avg_epoch_loss=3.19]

62it [00:04, 14.28it/s, epoch=1/20, validation_avg_epoch_loss=2.96]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.95it/s, epoch=2/20, avg_epoch_loss=2.6]

62it [00:04, 14.68it/s, epoch=2/20, validation_avg_epoch_loss=2.63]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.94it/s, epoch=3/20, avg_epoch_loss=2.45]

62it [00:04, 14.77it/s, epoch=3/20, validation_avg_epoch_loss=2.46]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50

In [23]:
valid_forecast_it, _ = make_evaluation_predictions(valid_ds, predictor=predictor, num_samples=100)
valid_predictions = {k: [] for k in SELECTED_PLANT_ID}
for forecast_entry in valid_forecast_it:
    valid_predictions[int(forecast_entry.item_id)].append(forecast_entry.mean_ts[-24:])

In [24]:
test_forecast_it, _ = make_evaluation_predictions(test_ds, predictor=predictor, num_samples=100)
test_predictions = {k: [] for k in SELECTED_PLANT_ID}
for forecast_entry in test_forecast_it:
    test_predictions[int(forecast_entry.item_id)].append(forecast_entry.mean_ts[-24:])

In [25]:
def calculate_wmape(preds, actuals):
    return np.sum(np.abs(preds-actuals)) / np.sum(np.abs(actuals))

for plant_id in SELECTED_PLANT_ID:
    preds = pd.concat(valid_predictions[plant_id])
    print(f"Valid WMAPE for {plant_id}")
    print(calculate_wmape(preds, valid_df[valid_df["rt_plant_id"] == plant_id].production[:len(preds)]))
    
    preds = pd.concat(test_predictions[plant_id])
    print(f"Test WMAPE for {plant_id}")
    print(calculate_wmape(preds, test_df[test_df["rt_plant_id"] == plant_id].production[:len(preds)]))

Valid WMAPE for 969
0.580001840139616
Test WMAPE for 969
0.6119181318390609
Valid WMAPE for 968
0.5340643261679394
Test WMAPE for 968
0.5266935374935354
Valid WMAPE for 1518
0.48484448397505775
Test WMAPE for 1518
0.6395204585850466
Valid WMAPE for 1484
0.736279828516832
Test WMAPE for 1484
0.6061906501361213
Valid WMAPE for 1507
0.5130290056893714
Test WMAPE for 1507
0.5849283748751638
Valid WMAPE for 1508
0.6964347166010888
Test WMAPE for 1508
0.620191834647525
Valid WMAPE for 1527
0.6685300537506824
Test WMAPE for 1527
0.6142981250719465
Valid WMAPE for 672
0.6835134624103687
Test WMAPE for 672
0.5865676882764479
Valid WMAPE for 1537
0.6351636061499323
Test WMAPE for 1537
0.6492668520982153
Valid WMAPE for 2180
0.6188401217002848
Test WMAPE for 2180
0.5115705812252234
Valid WMAPE for 1509
0.4731293196951392
Test WMAPE for 1509
0.6334974512648064
Valid WMAPE for 1506
0.6571291213929162
Test WMAPE for 1506
0.5772992927789427
Valid WMAPE for 1511
0.4890758898364198
Test WMAPE for 1511


In [26]:
calculate_wmape(
    pd.concat([pd.concat(valid_predictions[plant_id]) for plant_id in SELECTED_PLANT_ID]), 
    pd.concat([valid_df[valid_df["rt_plant_id"] == plant_id].production[:len(valid_predictions[SELECTED_PLANT_ID[0]])*24] for plant_id in SELECTED_PLANT_ID])
)

0.6102728146004286

In [27]:
calculate_wmape(
    pd.concat([pd.concat(test_predictions[plant_id]) for plant_id in SELECTED_PLANT_ID]), 
    pd.concat([test_df[test_df["rt_plant_id"] == plant_id].production[:len(test_predictions[SELECTED_PLANT_ID[0]])*24] for plant_id in SELECTED_PLANT_ID])
)

0.6004670615320927